<a href="https://colab.research.google.com/github/iisc-ts-group6/capstone/blob/main/SBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers pandas openpyxl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [ ]:
# Import necessary libraries
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
import re
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Load the Excel file
file_path = '/content/master_sheet.xlsx'
df = pd.read_excel(file_path)

In [ ]:
import re
# Define a text cleaning function
def clean_text(text):
    # Remove excess newline characters
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [ ]:
# Clean the data to ensure no NaN values
df.fillna("", inplace=True)

In [ ]:
df.head()
len(df)

695

In [ ]:
# Apply text cleaning to each column
df['question'] = df['Question'].apply(clean_text)
df['answer'] = df['Answer'].apply(clean_text)
df['D1'] = df['D1'].apply(clean_text)
df['D2'] = df['D2'].apply(clean_text)
df['D3'] = df['D3'].apply(clean_text)

In [ ]:
# Prepare training examples
train_examples = []

In [ ]:
# Iterate over each row to create positive and negative examples
for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']
    distractors = [row['D1'], row['D2'], row['D3']]

    # Positive example (question, correct answer)
    train_examples.append(InputExample(texts=[question, answer], label=1.0))

    # Negative examples (question, distractors)
    for distractor in distractors:
        train_examples.append(InputExample(texts=[question, distractor], label=0.0))

In [ ]:
# Split the dataset into training and testing sets
train_size = int(0.8 * len(train_examples))
test_size = len(train_examples) - train_size
train_dataset, test_dataset = random_split(train_examples, [train_size, test_size])

In [ ]:
# Convert the training examples into a DataLoader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=16)

In [ ]:
# Load pre-trained SBERT model
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Define the loss function for training
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 32.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [ ]:
import torch
# specify GPU
device = torch.device("cuda")

In [ ]:
# Train the SBERT model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)


Step,Training Loss


In [ ]:
# Evaluate the model on the test set
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(test_dataset, name='test')
model.evaluate(evaluator)

{'test_pearson_cosine': 0.9917285427103941,
 'test_spearman_cosine': 0.7463694130213747,
 'test_pearson_manhattan': 0.9908460673122494,
 'test_spearman_manhattan': 0.7463694130213747,
 'test_pearson_euclidean': 0.9910357095897013,
 'test_spearman_euclidean': 0.7463694130213747,
 'test_pearson_dot': 0.9917285424281427,
 'test_spearman_dot': 0.7463694130213747,
 'test_pearson_max': 0.9917285427103941,
 'test_spearman_max': 0.7463694130213747}

In [ ]:
# Example sentences to compute embeddings and evaluate similarity on real-time data
real_time_sentences = ["The quick brown fox jumps over the lazy dog", "A fast, brown fox leaps over a lazy dog"]


In [ ]:
# Clean the real-time sentences
real_time_sentences = [clean_text(sentence) for sentence in real_time_sentences]

In [ ]:
# Compute sentence embeddings
real_time_embeddings = model.encode(real_time_sentences)


In [ ]:
# Function to compute cosine similarity
def cosine_similarity(emb1, emb2):
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

In [ ]:
# Compute and print cosine similarity between real-time sentence pairs
for i in range(len(real_time_sentences)):
    for j in range(i+1, len(real_time_sentences)):
        sim = cosine_similarity(real_time_embeddings[i], real_time_embeddings[j])
        print(f"Similarity between '{real_time_sentences[i]}' and '{real_time_sentences[j]}': {sim:.4f}")

Similarity between 'the quick brown fox jumps over the lazy dog' and 'a fast brown fox leaps over a lazy dog': 0.9222


In [ ]:
# Function to test the model on new inputs
def test_similarity(question, correct_answer, student_answer):
    # Clean the input text
    question = clean_text(question)
    correct_answer = clean_text(correct_answer)
    student_answer = clean_text(student_answer)

    # Compute embeddings
    embeddings = model.encode([question, correct_answer, student_answer])

    # Compute similarity score between correct answer and student answer
    sim_score = cosine_similarity(embeddings[1], embeddings[2])

    return sim_score



In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00


In [ ]:
import os
f = open('/content/openapi_key_team.txt')
api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI

client = OpenAI()

# Function to generate feedback using GPT
def generate_feedback(question, correct_answer, student_answer):
    # Generate feedback using OpenAI GPT

    feedback_prompt = (
        f"The student was asked: '{question}'\n"
        f"The correct answer is: '{correct_answer}'\n"
        f"The student's answer is: '{student_answer}'\n"
        f"Explain why the student's answer is incorrect in 2 sentences. based on the {correct_answer}"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": feedback_prompt}
        ]
    )

    #feedback = response['choices'][0]['message']['content']
    feedback = response.choices[0].message.content
    return feedback

In [ ]:
# Example inputs
question = "What are the different types of crops?"
correct_answer = "The different types of crops include cereals, vegetables, and fruits."
student_answer = "The different types of crops include cereals and fruits"

# Test the model on the example inputs
similarity_score = test_similarity(question, correct_answer, student_answer)
print(f"Similarity score between the correct answer and the student answer: {similarity_score:.4f}")


# Check if the student answer is correct based on the threshold
if similarity_score > 0.80:
    print("Student answer is correct")
else:
    feedback = generate_feedback(question, correct_answer, student_answer)
    print("Student answer is incorrect. Feedback:")
    print(feedback)

Similarity score between the correct answer and the student answer: 0.9587
Student answer is correct


In [ ]:
question = "what does photosynthesis needs?"
correct_answer = "Photosynthesis needs carbon dioxide (CO2) and sunlight"
student_answer = "Photosynthesis needs carbon monoxide (CO) and sunlight"

# Test the model on the example inputs
similarity_score = test_similarity(question, correct_answer, student_answer)
print(f"Similarity score between the correct answer and the student answer: {similarity_score:.4f}")


# Check if the student answer is correct based on the threshold
if similarity_score > 0.98:
    print("Student answer is correct")
else:
    feedback = generate_feedback(question, correct_answer, student_answer)
    print("Student answer is incorrect. \n Feedback:")
    print(feedback)

Similarity score between the correct answer and the student answer: 0.8610
Student answer is incorrect. 
 Feedback:
The student's answer is incorrect because photosynthesis does not need carbon monoxide (CO) but actually requires carbon dioxide (CO2). Carbon dioxide is a colorless, odorless gas that plants absorb through tiny openings called stomata in their leaves. During photosynthesis, plants use carbon dioxide along with sunlight and water to produce glucose, a form of sugar that serves as their main source of energy. While carbon monoxide is also a colorless, odorless gas, it is toxic to plants as well as humans and animals. Therefore, carbon dioxide, not carbon monoxide, is an essential component for photosynthesis to occur.


In [ ]:
question = "what does photosynthesis needs?"
correct_answer = "Photosynthesis needs carbon dioxide (CO2) and sunlight"
student_answer = "Photosynthesis needs oxygen and carbon dioxide (CO2)"

# Test the model on the example inputs
similarity_score = test_similarity(question, correct_answer, student_answer)
print(f"Similarity score between the correct answer and the student answer: {similarity_score:.4f}")


# Check if the student answer is correct based on the threshold
if similarity_score > 0.98:
    print("Student answer is correct")
else:
    feedback = generate_feedback(question, correct_answer, student_answer)
    print("Student answer is incorrect. \n Feedback:")
    print(feedback)

Similarity score between the correct answer and the student answer: 0.9023
Student answer is incorrect. Feedback:
The student's answer is incorrect because photosynthesis actually requires carbon dioxide (CO2) and sunlight, not oxygen. During photosynthesis, plants use sunlight to convert carbon dioxide from the air into glucose (a type of sugar), releasing oxygen as a byproduct. Therefore, while both carbon dioxide and oxygen are involved in the process, it is carbon dioxide that is needed for photosynthesis to occur, not oxygen.


In [ ]:
question = "Why is humus important for soil health?"
correct_answer = "Humus is important for soil health because it provides organic matter and nutrients to the soil, improves soil structure and water retention capacity, and enhances microbial activity, which promotes overall plant growth and health."
student_answer = "Humus releases oxygen into the soil, creating a microclimate that requires more frequent watering."

# Test the model on the example inputs
similarity_score = test_similarity(question, correct_answer, student_answer)
print(f"Similarity score between the correct answer and the student answer: {similarity_score:.4f}")


# Check if the student answer is correct based on the threshold
if similarity_score > 0.98:
    print("Student answer is correct")
else:
    feedback = generate_feedback(question, correct_answer, student_answer)
    print("Student answer is incorrect. \n Feedback:")
    print(feedback)

Similarity score between the correct answer and the student answer: 0.5982
Student answer is incorrect. Feedback:
The student's answer is incorrect because it does not accurately explain why humus is important for soil health. Humus does not release oxygen into the soil to create a microclimate that requires more frequent watering. Instead, humus provides organic matter and nutrients to the soil, improves soil structure and water retention capacity, and enhances microbial activity. These qualities of humus contribute to better soil health, plant growth, and overall ecosystem functioning.


In [ ]:
question = "For any substance, why does the temperature remain constant during the change of state?"
correct_answer = "The temperature remains constant during a change of state because the heat energy being supplied is used to overcome the forces of attraction between particles, rather than increasing the temperature."
student_answer = "The heat energy supplied during a phase change isn't used to make the substance hotter. It's used to weaken the forces between particles, allowing them to rearrange into a new state."

# Test the model on the example inputs
similarity_score = test_similarity(question, correct_answer, student_answer)
print(f"Similarity score between the correct answer and the student answer: {similarity_score:.4f}")


# Check if the student answer is correct based on the threshold
if similarity_score > 0.98:
    print("Student answer is correct")
else:
    feedback = generate_feedback(question, correct_answer, student_answer)
    print("Student answer is incorrect. \n Feedback:")
    print(feedback)

Similarity score between the correct answer and the student answer: 0.7744
Student answer is incorrect. 
 Feedback:
The student's answer is incorrect because it does not directly address the concept that the temperature remains constant during a change of state, which is essential to understanding the phenomenon. Additionally, the statement could be misleading as it may suggest that the particles are physically rearranging into a new state, whereas it is more about the change in arrangement due to the weakening of intermolecular forces during a phase change.


In [ ]:
question = "what are oviparous animals?"
correct_answer = "Oviparous animals are animals that lay eggs which later develop into young ones outside the female body."
student_answer = "Animals that reproduce by laying eggs are oviparous. These eggs hatch outside the mother's body."

# Test the model on the example inputs
similarity_score = test_similarity(question, correct_answer, student_answer)
print(f"Similarity score between the correct answer and the student answer: {similarity_score:.4f}")


# Check if the student answer is correct based on the threshold
if similarity_score > 0.80:
    print("Student answer is correct")
else:
    feedback = generate_feedback(question, correct_answer, student_answer)
    print("Student answer is incorrect. \n Feedback:")
    print(feedback)

Similarity score between the correct answer and the student answer: 0.8582
Student answer is correct
